Note: there is an API but I need to spend time to understand it and to make it work. Some links, [here](https://docs.microsoft.com/en-us/partner-center/develop/get-all-azure-usage-analytics), [here](https://docs.microsoft.com/en-us/partner-center/insights-programmatic-analytics-api-get-report), and [here](https://docs.microsoft.com/en-us/partner-center/develop/partner-center-rest-api-reference). While I don't find time I will download the reports manually and process its data.

## Manual Procedure to dowload the data

1. Sign-in at https://partner.microsoft.com/
2. At Membership -> Competencies -> 'Download skill report'
3. At Insights -> Download Reports -> 'Azure Usage' (time frame 3 months)


In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [ ]:
%run 'docx_functions.ipynb'

In [ ]:
document_path = 'northwave_microsoft_partner_report.docx'

In [ ]:
create_file_based_template ('template/nw_style.docx',document_path)

## Processing Trainings

In [ ]:
df_skills = pd.read_csv('UserSkillsReport.csv',skiprows=4)

In [ ]:
df_skills_certified = df_skills[df_skills['Skill Type']=='Exam'][[' Name','Skill ID']].sort_values('Skill ID').reset_index(drop=True)
df_skills_certified

In [ ]:
search_n_replace_4dataframe (document_path, 'TABLE_CERTIFICATIONS', df_skills_certified, keep_anchor=False)

In [ ]:
df_summary_certifications = df_skills_certified['Skill ID'].value_counts().reset_index().rename({'index': 'Certification','Skill ID':'Count'}, axis=1)
df_summary_certifications

In [ ]:
search_n_replace_4dataframe (document_path, 'TABLE_SUMMARY_CERTIFICATIONS', df_summary_certifications, keep_anchor=False)

Keep in mind:
- Advanced Specialization on Cloud Security (6xAZ500 + 4SC200) [link](https://partner.microsoft.com/en-us/dashboard/mpn/program/security?program=HybridCloudSecurityASP)
- Advanced Specialization on Identity and Access Management (6xMS500)
- Advanced Specialization on Information Protection and Governance (6xMS500)
- Advanced Specialization on Threat Protection (6xMS500)

## Processing Azure Usage

In [ ]:
month = 'Feb-2022'

In [ ]:
df_azure_usage = pd.read_csv('Export_azure-usage_3Month_02242022_0805hrs_68b0fc37-ad4b-4c6e-ba22-3eaafe4b5e41.csv')

In [ ]:
df_azure_usage = df_azure_usage[df_azure_usage['CustomerName'] != 'Jair']

In [ ]:
df_azure_usage = df_azure_usage[df_azure_usage['Month']==month]

In [ ]:
df_azure_usage

In [ ]:
df_customer_resources = df_azure_usage['CustomerName'].value_counts().rename_axis('Customer').reset_index()
df_customer_resources = df_customer_resources.rename({'CustomerName': 'num_Resources'}, axis=1)
df_customer_resources = df_customer_resources.sort_values(by=['Customer']).reset_index(drop=True)
df_customer_resources

In [ ]:
search_n_replace_4dataframe (document_path, 'TABLE_CUSTOMER_LIST', df_customer_resources, keep_anchor=False)

## Customer, IndustryName and VerticalName

In [ ]:
df_customers_info = df_azure_usage.groupby(by=['CustomerName','IndustryName','VerticalName']).size().reset_index().drop(0,axis=1)
df_customers_info

In [ ]:
search_n_replace_4dataframe (document_path, 'TABLE_ CUSTOMER_INDUSTRY_VERTICAL', df_customers_info, keep_anchor=False)

In [ ]:
df_industry_vertical = df_customers_info.groupby(by=['IndustryName','VerticalName']).size().reset_index().rename({0: 'Count'}, axis=1)
# .reset_index(level=[0,1])
df_industry_vertical

In [ ]:
search_n_replace_4dataframe (document_path, 'TABLE_INDUSTRY_VERTICAL', df_industry_vertical, keep_anchor=False)

## Customer usage with different UsageUnits

In [ ]:
df_usage_extended = df_azure_usage.groupby(by=['CustomerName','UsageUnits']).agg({'UsageQuantity':[lambda x: x.astype(int).sum()]}).reset_index()
df_usage_extended

In [ ]:
search_n_replace_4dataframe (document_path, 'TABLE_CUSTOMER_USAGE_EXTENDED', df_usage_extended, keep_anchor=False)

In [ ]:
customer_total_usage = df_azure_usage.groupby(by=['CustomerName']).agg({'UsageQuantity':['sum']}).reset_index()
customer_total_usage

In [ ]:
search_n_replace_4dataframe (document_path, 'TABLE_CUSTOMER_USAGE_SUM', customer_total_usage, keep_anchor=False)

In [ ]:
sum_total_usage_GB = "{:,}".format(df_azure_usage['UsageQuantity'].sum())
sum_total_usage_GB

In [ ]:
search_n_replace_4word (document_path, 'TEXT_SUM_TOTAL_USAGE', sum_total_usage_GB)

In [ ]:
!open $document_path